In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Wczytanie danych
# df = pd.read_csv('dataset-tickets-multi-lang-4-20k.csv')

df = pd.read_csv('de-tickets.csv')

In [2]:



# 2. Łączenie kolumn subject i body w jeden tekst
df['text'] = df['subject'].fillna('') + ' ' + df['body'].fillna('')

# 3. Usuwamy wiersze z brakującym typem
df = df.dropna(subset=['type'])

# 4. Tokenizacja tekstu
max_words = 20000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# 5. Kodowanie etykiet
le = LabelEncoder()
labels = le.fit_transform(df['type'])
labels_cat = to_categorical(labels)

# 6. Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(padded, labels_cat, test_size=0.2, random_state=42)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 7. Budowa modelu LSTM
model = Sequential()
model.add(Embedding(max_words, 256, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(labels_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 8. Trening
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# 9. Ewaluacja
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.2f}")


Epoch 1/50
122/122 [==============================] - 54s 418ms/step - loss: 0.7972 - accuracy: 0.6521 - val_loss: 0.5134 - val_accuracy: 0.7677
Epoch 2/50
122/122 [==============================] - 51s 417ms/step - loss: 0.4463 - accuracy: 0.7891 - val_loss: 0.4826 - val_accuracy: 0.7698
Epoch 3/50
122/122 [==============================] - 52s 425ms/step - loss: 0.3035 - accuracy: 0.8653 - val_loss: 0.5452 - val_accuracy: 0.7646
Epoch 4/50
122/122 [==============================] - 52s 429ms/step - loss: 0.1633 - accuracy: 0.9409 - val_loss: 0.7027 - val_accuracy: 0.7297
Epoch 5/50
122/122 [==============================] - 53s 434ms/step - loss: 0.1065 - accuracy: 0.9630 - val_loss: 0.9698 - val_accuracy: 0.6547
Epoch 6/50
122/122 [==============================] - 53s 436ms/step - loss: 0.0654 - accuracy: 0.9776 - val_loss: 0.9400 - val_accuracy: 0.7657
Epoch 7/50
122/122 [==============================] - 53s 434ms/step - loss: 0.0237 - accuracy: 0.9925 - val_loss: 0.9897 - val_ac